In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install adafruit-io

In [ ]:
import pandas as pd

from Adafruit_IO import RequestError, Client, Feed
import time
import heapq
import queue


Adafruit_IO_USER = 'Jiwon1121'
Adafruit_IO_KEY = 'aio_vPev73gZSpASX8arSEyZ0bik3Hc4'

aio = Client(Adafruit_IO_USER,Adafruit_IO_KEY)

feeds = aio.feeds()

firename = []
smokename = []
ledname = []


firekey = []
smokekey = []
ledkey = []

class Map (object):
    def __init__(self):
        self.fire_place_num = []
        self.exit = [1, 3]  # 탈출구인 노드 num 들
        self.all_node_num = 12
        self.node = []  # 모든 노드들
        self.check = []
        self.all_node = []  # 모든 노드번호들 넣을곳 추후 탈출구 노드를 제외하고 화살표 방향을 정할때 씀
        self.now_distance = []  # 방향구하는거 쓸떄 비교에 쓰임

        self.initiation_smoke_values = []   # 각 노드별 초기 연기 데이터 값 


knu = Map()
count = -1


for i in range(0,12): #노드 개수로 바꾸기
      
    fires = 'node{0:>02d}.fire{0:>02d}'.format(i,i)
    smokes = 'node{0:>02d}.smoke{0:>02d}'.format(i,i)
    leds = 'node{0:>02d}.led{0:>02d}'.format(i,i)
    
    firekey.append(fires)
    smokekey.append(smokes)
    ledkey.append(leds)

    name_fire = 'fire{0:>02d}'.format(i,i)
    name_smoke = 'smoke{0:>02d}'.format(i,i)
    name_led = 'led{0:>02d}'.format(i,i)

    firename.append(fires)
    smokename.append(smokes)
    ledname.append(leds)
    
    globals()['fire{0:>02d}'.format(i,i)] = aio.feeds(fires) #연결
    globals()['smoke{0:>02d}'.format(i,i)] = aio.feeds(smokes)
    globals()['led{0:>02d}'.format(i,i)] = aio.feeds(leds)



class Node (object):
    def __init__(self, num):
        # self.location = np.array([x,y,z])
        self.index = num
        self.forward= None
        self.backward = None
        self.left = None
        self.right = None
        self.up = None
        self.down = None
        self.distance = float('inf')
        self.temp_distance = 0
        self.direction = [0, 0, 0, 0, 0, 0] #상, 하, 좌, 우, 위, 아래
        self.exit_diret = 0 # <- 자료형 뭐고?
        self.exit_diret_num = []
        self.visited_place = []
    def set_index(self, num):
        self.index = num

    def set_distance(self, num):
        if(self.distance > num):
            self.distance=num
    def set_exit_diret_num(self, list):
        self.exit_diret_num = list

def AddNode(): #count는 node num을 지정해주기 위해 사용
    global count
    count += 1
    return Node(count)

def find_linked_node(Node): # 입력 Node와 연결된 노드 Num을 index_list에 넣음
    dir = Node.direction
    index_list = []
    if dir[0] == 1:
        index_list.append(Node.forward.index)
    if dir[1] == 1:
        index_list.append(Node.backward.index)
    if dir[2] == 1:
        index_list.append(Node.left.index)
    if dir[3] == 1:
        index_list.append(Node.right.index)
    if dir[4] == 1:
        index_list.append(Node.up.index)
    if dir[5] == 1:
        index_list.append(Node.down.index)
    return index_list

def dijkstra(start,linked_node_list): # 모든 노드간 가중치가 1, 이는 추후 바꿀수 있으며 모든 가중치가 1이기에 출발 노드에서 모든 노드까지의 거리 측정가능
    distances = {node: float('inf') for node in range(84)}
    distances[start] = 0

    queue = []
    heapq.heappush(queue, [distances[start], start])

    while queue:
        current_distance, current_node = heapq.heappop(queue)

        if distances[current_node] < current_distance:
            continue

        for adjacent in linked_node_list[current_node]:
            distance = current_distance + 1

            if distance < distances[adjacent]:
                distances[adjacent] = distance
                heapq.heappush(queue, [distance, adjacent])
    return distances

def set_weight(node,exit):# 다익스트라를 통해 탈출구에서 모든 노드까지의 거리를 구한후 비교해서 최솟값 넣기
    for e in exit:
        result = dijkstra(e, linked_node_list)
        for i in range(knu.all_node_num):
            node[i].set_distance(result[i])

def fire_test(fire_node_num, send_node_num1, send_node_num2, node, linked_node_list):
    print("fire test")
    queue = []
    del_list = []
    fire_start = fire_node_num
    send1 = send_node_num1
    send2 = send_node_num2
    node[fire_node_num].distance = float('inf')

    if fire_node_num not in knu.fire_place_num:
        knu.fire_place_num.append(fire_node_num)
    for i in linked_node_list[send2]:
        if i != send1 and i not in knu.exit: #not in fire_place_num:
            queue.insert(0,i)

    for i in queue:
        substract = []
        node_list = []
        shortest_node_num = []
        temp_exit_diret = 0
       
        for j in linked_node_list[i]:
            node_list.append(j)
           
            if j != send2:
                substract.append(node[j].distance - node[i].distance)
               
            else:
                substract.append(float('inf'))
        
        #print(node_list)
        shortest_distance_node_index = [i for i, value in enumerate(substract) if value == min(substract)]
        temp_exit_diret = node[i].exit_diret

        if (min(substract) != -1 or node[i].exit_diret_num != shortest_distance_node_index) and min(substract) != float('inf'):
            node[i].exit_diret = 0
          
          
            if send2 not in node[i].visited_place:
                node[i].visited_place.append(send2)
                print('7')
            #node[i].distance += 2

            node[i].set_exit_diret_num(shortest_distance_node_index)
            for k in shortest_distance_node_index:
                shortest_node_num.append(node_list[k])
             
            
          
            if node[i].forward != None and node[i].forward.index in shortest_node_num:
                if node[i].forward.index not in node[i].visited_place:
                    node[i].exit_diret += 32
               
            if node[i].backward != None and node[i].backward.index in shortest_node_num :
                if node[i].backward.index  not in node[i].visited_place:
                    node[i].exit_diret += 16
            if node[i].right != None and node[i].right.index in shortest_node_num :
                if node[i].right.index  not in node[i].visited_place:
                    node[i].exit_diret += 4
               
            if node[i].left != None and node[i].left.index in shortest_node_num:
                if node[i].left.index not in node[i].visited_place:
                    node[i].exit_diret += 8
            if node[i].up != None and node[i].up.index in shortest_node_num :
                if node[i].up.index not in node[i].visited_place:
                    node[i].exit_diret += 2
            if node[i].down != None and node[i].down.index in shortest_node_num:
                if node[i].down.index not in node[i].visited_place:
                    node[i].exit_diret += 1

        else:
            del_list.append(i)
          
    
    for i in del_list:
        queue.remove(i)
       
    for i in queue:
        fire_test(fire_start,send2,i,node,linked_node_list)
        

def create_sensor_map(length_file, width_file, stairs_file):
    """센서의 고유번호, 관계, 위치가 저장된 파일을 전달하면 그래프화 하고 sensor_map에 저장."""
    sensor_map = []
    file_list = [length_file, width_file, stairs_file]

    for file_name in file_list:

        with open(file_name) as sensor_file:

            for line in sensor_file:

                # 파일 입력받을 때 스플릿
                raw_data = line.strip().split("-")
                prev_sensor = None

                for sensor_num in raw_data:
                    sensor_num = int(sensor_num.strip())

                    if sensor_num not in sensor_map:  # 센서 정보가 센서 딕셔너리에 없으면
                        current_sensor = knu.node[sensor_num]
                        sensor_map.append(sensor_num)
                    else:
                        current_sensor = knu.node[sensor_num]
                    # print(current_sensor)
                    # print(prev_sensor)
                    # if file_name == "exit.txt":  # 출구 센서 지정
                    #     current_sensor.state = 'E'
                    #     exit_node[sensor_num] = current_sensor

                    if prev_sensor is not None:
                        if file_name == "/content/gdrive/My Drive/file01/width.txt":  # 가로 연결된 센서
                            current_sensor.left = prev_sensor
                            prev_sensor.right = current_sensor
                            current_sensor.direction[2] = 1
                            prev_sensor.direction[3] = 1
                        elif file_name == "/content/gdrive/My Drive/file01/length.txt":  # 세로 연결된 센서
                            current_sensor.forward = prev_sensor
                            prev_sensor.backward = current_sensor
                            current_sensor.direction[0] = 1
                            prev_sensor.direction[1] = 1
                        elif file_name == "/content/gdrive/My Drive/file01/stairs.txt":  # 계단 센서
                            current_sensor.down = prev_sensor
                            prev_sensor.up = current_sensor
                            current_sensor.direction[5] = 1
                            prev_sensor.direction[4] = 1
                            # 계단 weight = 2로 설정
                            # current_sensor.weight['down'] = 2
                            # prev_sensor.weight['up'] = 2

                    prev_sensor = current_sensor

def find_exit(all_node,node, linked_node_list):
    for i in all_node:
        substract = []
        shortest_node_num = []
        for j in linked_node_list[i]:
            substract.append(node[j].distance - node[i].distance)
        shortest_distance_node_index = [i for i, value in enumerate(substract) if value == min(substract)]
        node[i].set_exit_diret_num(shortest_distance_node_index)
        for k in shortest_distance_node_index:
            shortest_node_num.append(linked_node_list[i][k]) # output = shortest node_num
            #print("aa->",shortest_node_num)
        if node[i].forward != None and node[i].forward.index in shortest_node_num:
            node[i].exit_diret += 32
        if node[i].backward != None and node[i].backward.index in shortest_node_num:
            node[i].exit_diret += 16
        if node[i].right != None and node[i].right.index in shortest_node_num:
            node[i].exit_diret += 4
        if node[i].left != None and node[i].left.index in shortest_node_num:
            node[i].exit_diret += 8
        if node[i].up != None and node[i].up.index in shortest_node_num:
            node[i].exit_diret += 2
        if node[i].down != None and node[i].down.index in shortest_node_num:
            node[i].exit_diret += 1



if __name__ == "__main__":
    

    for i in range(knu.all_node_num):
        knu.node.append(AddNode())




    create_sensor_map('/content/gdrive/My Drive/file01/length.txt', '/content/gdrive/My Drive/file01/width.txt', '/content/gdrive/My Drive/file01/stairs.txt')



    #다익스트라 알고리즘

    #linked_node_list[인접노드를 찾을 노드 num] = [인접 node num들]
    linked_node_list = []
    for i in range(knu.all_node_num):
        linked_node_list.append([])
        for j in find_linked_node(knu.node[i]):
            linked_node_list[i].append(j)




    set_weight(knu.node, knu.exit) # 모든 노드에서 가장 가까운 탈출구까지 거리 구하는 함수

        
 
    
    for i in range(knu.all_node_num): # 각 노드들이 가진 거리들 출력
        knu.now_distance.append(knu.node[i].distance)

    #화재전 노드 방향 연결
    for i in range(knu.all_node_num):
        knu.all_node.append(i)
        
    for j in knu.exit:
        knu.all_node.remove(j)

    find_exit(knu.all_node, knu.node, linked_node_list)

    '''
    for i in range(knu.all_node_num):
        print(i,'->',knu.node[i].exit_diret)
    #print("node21 visited_place",node[21].visited_place)
    '''
    #for i in range(0, 12):
    #    print(knu.node[i].exit_diret)


    '''
    fire_test(7,7,7,map.node,linked_node_list)
    print("fire in 7")
    for i in range(map.all_node_num):
        print(i, '->', map.node[i].exit_diret)
    # #print("node21 visited_place", node[21].visited_place)
    
    fire_test(11,11,11,map.node,linked_node_list)
    print("fire in 18")
    for i in range(map.all_node_num):
        print(i, '->', map.node[i].exit_diret)
    #print("node21 visited_place", node[21].visited_place)
        
    fire_test(3, 3, 3, map.node, linked_node_list)
    print("fire in 20")
    for i in range(map.all_node_num):
        print(i, '->', map.node[i].exit_diret)
    # #print("node21 visited_place", node[21].visited_place)

    fire_test(16, 16, 16, map.node, linked_node_list)
    print("fire in 20")
    for i in range(map.all_node_num):
        print(i, '->', map.node[i].exit_diret)


    print(map.fire_place_num)
    # #print("node21 visited_place", node[21].visited_place)
    '''
    led_num = 0

    for led_name in ledkey: # 알고리즘으로 계산한 방향을 adafruit로 보내는 반복문
        led = aio.feeds(led_name)
        aio.send_data(led.key, knu.node[led_num].exit_diret)
        print("knu.node num is")
        print(knu.node[led_num].exit_diret)
        print("first led_name is ")
        print(led_name)
        time.sleep(2)
        led_num += 1

    firesensor_value = []   # 이미 감지했던 노드들
    smokesensor_value = []  # 이미 감지했던 노드들

    for feed in smokekey:
        smoke_value = int(aio.receive(feed).value)
        knu.initiation_smoke_values.insert(i, smoke_value)


    
    """데이터 받아온다"""
    while(1) :
        print("while")
        for firesensor, smokesensor in zip(firekey, smokekey) :
            
            #fire_value = int(float(aio.receive(firesensor).value))
            fire_value = 0
            smoke_value = int(float(aio.receive(smokesensor).value))
           

            firenode_number = int(firesensor[-2:])   #이 변수에 저장되는 게 해당 노드 번호임
             
            if fire_value >= 60 : # 불이 났을 때 -> 1말고 
                               
                node_num = firenode_number

                
                """불 감지하면 노드 번호를 firenode_value 리스트에 넣음"""
                if node_num not in knu.firesensor_value:
                    firesensor_value.append(node_num) #노드 중복 방지
                    """값 받고 알고리즘 실행"""
                    fire_test(node_num, node_num, node_num, knu.node, linked_node_list)

                    aio.send_data(led00.key, knu.node[0].exit_diret)
                    print("knu.node num is")
                    print(knu.node[0].exit_diret)
                    aio.send_data(led01.key, knu.node[1].exit_diret)
                    print("knu.node num is")
                    print(knu.node[1].exit_diret)
                    aio.send_data(led02.key, knu.node[2].exit_diret)
                    print("knu.node num is")
                    print(knu.node[2].exit_diret)
                    aio.send_data(led03.key, knu.node[3].exit_diret)
                    print("knu.node num is")
                    print(knu.node[3].exit_diret)
                    aio.send_data(led04.key, knu.node[4].exit_diret)
                    print("knu.node num is")
                    print(knu.node[4].exit_diret)
                    aio.send_data(led05.key, knu.node[5].exit_diret)
                    print("knu.node num is")
                    print(knu.node[5].exit_diret)
                    aio.send_data(led06.key, knu.node[6].exit_diret)
                    print("knu.node num is")
                    print(knu.node[6].exit_diret)
                    aio.send_data(led07.key, knu.node[7].exit_diret)
                    print("knu.node num is")
                    print(knu.node[7].exit_diret)
                    aio.send_data(led08.key, knu.node[8].exit_diret)
                    print("knu.node num is")
                    print(knu.node[8].exit_diret)
                    aio.send_data(led09.key, knu.node[9].exit_diret)
                    print("knu.node num is")
                    print(knu.node[9].exit_diret)
                    aio.send_data(led10.key, knu.node[10].exit_diret)
                    print("knu.node num is")
                    print(knu.node[10].exit_diret)
                    aio.send_data(led11.key, knu.node[11].exit_diret)
                    print("knu.node num is")
                    print(knu.node[11].exit_diret)

                    '''
                    결과값 출력하는 구문
                    print("fire in", node_num)
                    for i in range(map.all_node_num):
                        print(i, '->', map.node[i].exit_diret)
                    '''
                    
                    firesensor_value.append(firenode_number)
                    
                
            smokenode_number = int(smokesensor[-2:])#이 변수에 저장되는 게 해당 노드 번호임
      
            if(smoke_value - int(knu.initiation_smoke_values[smokenode_number]) >= 35) : 
                #led_feed = smokesensor.replace("smoke", "led", 1)
                  

                """연기 감지하면 노드 번호를 smokesensor_value 리스트에 넣음"""
                node_num = smokenode_number #불이 난 노드

                print("first node_num is")
                print(node_num)
                print("first smokesensor_value is")
                print(smokesensor_value)
                if node_num not in smokesensor_value:
                    smokesensor_value.append(node_num) #노드 중복 방지
                    print("first smokesensor_value is")
                    print(smokesensor_value)

                    """값 받고 알고리즘 실행"""
                    fire_test(node_num, node_num, node_num, knu.node, linked_node_list)



                # 알고리즘으로 계산한 방향을 adafruit로 보내기
                #led = aio.feeds(led_name) #feed 연결 -> 밖에서 하는 방법?
                #aio.send_data(led.key, knu.node[led_num].exit_diret)
                aio.send_data(led00.key, knu.node[0].exit_diret)
                print("knu.node num is")
                print(knu.node[0].exit_diret)
                aio.send_data(led01.key, knu.node[1].exit_diret)
                print("knu.node num is")
                print(knu.node[1].exit_diret)
                aio.send_data(led02.key, knu.node[2].exit_diret)
                print("knu.node num is")
                print(knu.node[2].exit_diret)
                aio.send_data(led03.key, knu.node[3].exit_diret)
                print("knu.node num is")
                print(knu.node[3].exit_diret)
                aio.send_data(led04.key, knu.node[4].exit_diret)
                print("knu.node num is")
                print(knu.node[4].exit_diret)
                aio.send_data(led05.key, knu.node[5].exit_diret)
                print("knu.node num is")
                print(knu.node[5].exit_diret)
                aio.send_data(led06.key, knu.node[6].exit_diret)
                print("knu.node num is")
                print(knu.node[6].exit_diret)
                aio.send_data(led07.key, knu.node[7].exit_diret)
                print("knu.node num is")
                print(knu.node[7].exit_diret)
                aio.send_data(led08.key, knu.node[8].exit_diret)
                print("knu.node num is")
                print(knu.node[8].exit_diret)
                aio.send_data(led09.key, knu.node[9].exit_diret)
                print("knu.node num is")
                print(knu.node[9].exit_diret)
                aio.send_data(led10.key, knu.node[10].exit_diret)
                print("knu.node num is")
                print(knu.node[10].exit_diret)
                aio.send_data(led11.key, knu.node[11].exit_diret)
                print("knu.node num is")
                print(knu.node[11].exit_diret)
        time.sleep(25)      
            



knu.node num is
4
first led_name is 
node00.led00
knu.node num is
0
first led_name is 
node01.led01
knu.node num is
8
first led_name is 
node02.led02
knu.node num is
0
first led_name is 
node03.led03
knu.node num is
16
first led_name is 
node04.led04
knu.node num is
16
first led_name is 
node05.led05
knu.node num is
16
first led_name is 
node06.led06
knu.node num is
16
first led_name is 
node07.led07
knu.node num is
16
first led_name is 
node08.led08
knu.node num is
16
first led_name is 
node09.led09
knu.node num is
28
first led_name is 
node10.led10
knu.node num is
16
first led_name is 
node11.led11
while
while
first node_num is
6
first smokesensor_value is
[]
first smokesensor_value is
[6]
fire test
7
fire test
knu.node num is
4
knu.node num is
0
knu.node num is
8
knu.node num is
0
knu.node num is
16
knu.node num is
16
knu.node num is
16
knu.node num is
16
knu.node num is
16
knu.node num is
16
knu.node num is
12
knu.node num is
16
while
first node_num is
1
first smokesensor_value is


ThrottlingError: ignored